In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import zipfile
import random
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from skimage import io
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# extract all contents of zip file
train_path = "../input/dogs-vs-cats/train.zip"
zip_train = zipfile.ZipFile(train_path, 'r')
zip_train.extractall("./")
zip_train.close()

In [ ]:
# now we have a folder called train in our root dir
#let's find the contents of the folder
train_img_names = os.listdir("./train")
train_img_names[0:10]

In [ ]:
len(train_img_names)

In [ ]:
img = random.choice(train_img_names)
plt.imshow(plt.imread("./train/"+img))

In [ ]:
print(plt.imread("./train/"+img).shape)

In [ ]:
# preparing training data
categories = []
for img in train_img_names:
    if img.split(".")[0] == "dog":
        categories.append(1)
    else:
        categories.append(0)

In [ ]:
df = pd.DataFrame({
    'filename': train_img_names,
    'category': categories
})

In [ ]:
df.head()

In [ ]:
df['category'].value_counts().plot.bar()

In [ ]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.annotations = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)
        

In [ ]:
# hyper parameters
in_channel = 3
num_classes = 2
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [ ]:
data_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.ColorJitter(),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [ ]:
# Load data
dataset = CatsAndDogsDataset(df = df, root_dir = "./train", transform = data_transform)
train_set, val_set = torch.utils.data.random_split(dataset, [20000, 5000])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_set)

In [ ]:
samples, labels = iter(train_loader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))
print(samples[0].shape)

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features

In [ ]:
model.fc = nn.Linear(num_ftrs,2)
model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the network
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda
        data = data.to(device)
        targets = targets.to(device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad() #set all gradients to zero for each batch
        loss.backward()

        # gradient descend or adam step
        optimizer.step()

In [ ]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [ ]:
check_accuracy(train_loader, model)

In [ ]:
check_accuracy(val_loader, model)